In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd
import re

# URL base
parte_url = 'https://one.bees.com/price-admin/view-prices?sortBy=productTitle&sortAs=ASC&page={page}&search=&selectedCountry=BR&selectedVendor=c9a6bf31-63bf-4f13-bfef-32c850c6cff6&ddc='
filiais = ['CASTASSP', 'CASTASRJ', 'CASTAES','CASTASMG']

def fazer_login(navegador, usuario, senha):
    """Função para fazer login no site."""
    navegador.get('https://one.bees.com/auth/login')
    sleep(5)
    navegador.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="signInName"]').send_keys(usuario)
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="next"]').click()
    sleep(5)
    navegador.find_element(By.XPATH, '//*[@id="password"]').send_keys(senha)
    sleep(1)
    navegador.find_element(By.XPATH, '//*[@id="next"]').click()
    sleep(5)
    navegador.find_element(By.XPATH, '//*[@id="single-spa-application:@admin-portal/main-mfe"]/section/header/div[1]/div/div[1]/div[3]/div[2]/button').click()
    sleep(5)

def limpar_valor(valor):
    """Remove o prefixo 'R$' e espaços, e mantém apenas o número com a vírgula."""
    if valor:
        return re.sub(r'[^\d,]', '', valor)
    return ''

def raspar_dados(navegador, url_base, filiais):
    """Função para raspar os dados das páginas de preços para as filiais especificadas."""
    todos_os_produtos = []
    for filial in filiais:
        pagina = 1
        while True:
            url = url_base.format(page=pagina) + filial
            navegador.get(url)
            sleep(10)
            dados = BeautifulSoup(navegador.page_source, 'html.parser')
            
            # Verificar se a página está vazia
            if dados.find('span', attrs={'data-testid': 'prices_table_empty_title'}):
                print(f'Nenhum preço encontrado para a filial: {filial} na página {pagina}')
                break

            # Encontrar todas as linhas da tabela
            linhas = dados.find_all('tr')
            
            if not linhas or len(linhas) <= 1:
                # Se não houver linhas ou se a página estiver vazia, interrompe o loop
                break

            for linha in linhas[1:]:  # Ignorar o cabeçalho da tabela
                celulas = linha.find_all('td', attrs={'role': 'cell'})
                
                # Verificar a quantidade de células na linha
                total_celulas = len(celulas)
                if total_celulas >= 9:
                    produto = {
                        'Seleçao': celulas[0].get_text(strip=True),
                        'Descrição': celulas[1].get_text(strip=True),
                        'Código': celulas[2].get_text(strip=True),
                        'Matriz': celulas[3].get_text(strip=True),
                        'Estado': celulas[4].get_text(strip=True),
                        'Marca': celulas[5].get_text(strip=True),
                        'Preço': limpar_valor(celulas[6].get_text(strip=True)).replace('"',''),
                        'Desconto Normal': limpar_valor(celulas[7].get_text(strip=True)),
                        'Desconto Gradual': limpar_valor(celulas[8].get_text(strip=True))
                    }
                    todos_os_produtos.append(produto)

            pagina += 1  # Passar para a próxima página
    return todos_os_produtos

# Main
if __name__ == "__main__":
    # Dados de login
    usuario = 'leonardo.campos@rigarr.com.br'
    senha = 'Br@sil32aaaaaaa'
    filial = 'Castas'

    # Inicializar o navegador
    navegador = webdriver.Chrome()
    navegador.maximize_window()
    sleep(10)
    try:
        # Fazer login
        fazer_login(navegador, usuario, senha)

        # Raspagem de dados
        produtos = raspar_dados(navegador, parte_url, filiais)

        # Criar DataFrame do pandas
        df = pd.DataFrame(produtos)

        # Mostrar o DataFrame como uma tabela
        display(df)

        # Salvar o DataFrame em um arquivo CSV com codificação UTF-8
        df.to_csv(f'produtosbees_{filial}.csv', index=False, encoding='utf-8-sig')
    finally:
        # Fechar o navegador
        print("Acabou")
        navegador.quit()
        


Nenhum preço encontrado para a filial: CASTASSP na página 8
Nenhum preço encontrado para a filial: CASTASRJ na página 8
Nenhum preço encontrado para a filial: CASTAES na página 8
Nenhum preço encontrado para a filial: CASTASMG na página 8


,Seleçao,Descrição,Código,Matriz,Estado,Marca,Preço,Desconto Normal,Desconto Gradual
0,,Espumante Espumante Alud Branco 750Ml,3011,CASTASSP,Habilitado,,"35,90",,
1,,Espumante Espumante Alud Rose 750Ml,3063,CASTASSP,Habilitado,,"35,90",,
2,,Vinho Adega Da Serra Tinto 750Ml,2914,CASTASSP,Desabilitado,,"32,90",,
3,,Vinho Adele Branco 750Ml,2917,CASTASSP,Habilitado,,"56,90",,
4,,Vinho Adele Rose 750Ml,2916,CASTASSP,Habilitado,,"56,90",,
...,...,...,...,...,...,...,...,...,...
239,,Vinho Villa Rosa Rose Escolha 750Ml,3025,CASTASMG,Desabilitado,,,,
240,,Vinho Villa Rosa Tinto 750Ml,2614,CASTASMG,Habilitado,,"97,70",,
241,,Vinho Villa Rosa Tinto Escolha 750Ml,3024,CASTASMG,Desabilitado,,,,
242,,Vinho Villa Rosa Tinto Privado 750Ml,3027,CASTASMG,Desabilitado,,,,


Acabou
